# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import re

from collections import Counter

import pandas as pd

## Read data and perform required transformations

In [3]:
ROOT_DIR = '/content/drive/MyDrive/NLP_Innopolis'

In [4]:
BIG_PATH = os.path.join(ROOT_DIR, 'big.txt')

BIGRAMS_PATH = os.path.join(ROOT_DIR, 'bigrams.txt')
FIVEGRAMS_PATH = os.path.join(ROOT_DIR, 'fivegrams.txt')
COCA_PATH = os.path.join(ROOT_DIR, 'coca_all_links.txt')

In [5]:
encoding = 'latin-1'

In [6]:
# Read the data into a DataFrame using pandas
bigrams_df = pd.read_csv(BIGRAMS_PATH, sep='\t', header=None, names=['count', 'first_word', 'second_word'], encoding=encoding)
coca_df = pd.read_csv(COCA_PATH, sep='\t', header=None, names=['count', 'first_word', 'second_word', 'part_of_speech1', 'part_of_speech2'], encoding=encoding)

fivegrams_df = pd.read_csv(FIVEGRAMS_PATH, sep='\t', header=None, names=['count', 'first_word', 'second_word', 'third_word', 'fourth_word', 'fifth_word'], encoding=encoding)

coca_df = coca_df.drop(columns=['part_of_speech1', 'part_of_speech2'])

In [7]:
bigrams_df.fillna('null', inplace=True)
coca_df.fillna('null', inplace=True)
fivegrams_df.fillna('null', inplace=True)

In [8]:
bigrams_df.sample(5)

,count,first_word,second_word
246456,23,different,bands
502800,25,look,uncomfortable
659401,98,photographers,in
839244,181,that,managed
753280,52,serious,opposition


In [9]:
coca_df.sample(5)

,count,first_word,second_word
139526,31,sophomore,center
78344,29,self-made,men
19926,45,disadvantaged,backgrounds
108948,43,data,bus
75573,73,remaining,members


In [10]:
fivegrams_df.sample(5)

,count,first_word,second_word,third_word,fourth_word,fifth_word
811530,6,the,whole,day,on,the
66558,18,and,i,wonder,if,it
914156,9,trying,to,figure,out,when
613826,7,outside,the,united,states,was
147961,10,been,slow,to,respond,to


In [11]:
# Extract ngrams from both DataFrames
bigrams_dict = dict(zip((bigrams_df['first_word'] + ' ' + bigrams_df['second_word']).str.lower(), bigrams_df['count']))
coca_dict = dict(zip((coca_df['first_word'] + ' ' + coca_df['second_word']).str.lower(), coca_df['count']))

fivegrams_dict = dict(zip((fivegrams_df['first_word'] + ' ' + fivegrams_df['second_word'] + ' ' + fivegrams_df['third_word'] + ' ' + fivegrams_df['fourth_word'] + ' ' + fivegrams_df['fifth_word']).str.lower(), fivegrams_df['count']))

bigrams_counter = Counter(bigrams_dict)
coca_counter = Counter(coca_dict)
fivegrams_counter = Counter(fivegrams_dict)

# Combine bigrams from both counters
bigrams_counter = bigrams_counter + coca_counter

In [12]:
all_bigram_words = {word for bigram in bigrams_counter.keys() for word in bigram.split()}
all_fivegram_words = {word for fivegram in fivegrams_counter.keys() for word in fivegram.split()}

all_words = {}

bigram_values_sum = sum(bigrams_counter.values())
fivegram_values_sum = sum(fivegrams_counter.values())

## Norvig spelling correction implementation

In [15]:
def norvig_words(text): return re.findall(r'\w+', text.lower())

NORVIG_WORDS = Counter(norvig_words(open(BIG_PATH).read()))

def norvig_P(word, N=sum(NORVIG_WORDS.values())):
    "Probability of `word`."
    return NORVIG_WORDS[word] / N

def norvig_correction(word):
    "Most probable spelling correction for word."
    return max(norvig_candidates(word), key=norvig_P)

def norvig_candidates(word):
    "Generate possible spelling corrections for word."
    return (norvig_known([word]) or norvig_known(norvig_edits1(word)) or norvig_known(norvig_edits2(word)) or [word])

def norvig_known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in NORVIG_WORDS)

def norvig_edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def norvig_edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in norvig_edits1(word) for e2 in norvig_edits1(e1))

def norvig_correct_text(text):
  words = text.split()
  corrected_words = [norvig_correction(word) for word in words]
  return ' '.join(corrected_words)

In [16]:
norvig_correct_text('hdw c0n i0 not truot you if i sincerly lovee dyou')

'how can in not trust you if i sincerely love you'

## Ngram spelling correction implementation

In [36]:
def get_ngram_P(n = 2):
  """
  This function creates a closure that returns the probability of an n-gram.

  Args:
      n (int, optional): The n-gram size (defaults to 2).

  Returns:
      function: A function that takes an n-gram as input and returns its probability.
  """
  def P(ngram):
    """
    Calculates the probability of an n-gram based on pre-computed counts.

    Args:
        ngram (str): The n-gram to calculate the probability for.

    Returns:
        float: The probability of the n-gram.
    """
    if n == 2:
      counter = bigrams_counter
      N = bigram_values_sum
    elif n == 5:
      counter = fivegrams_counter
      N = fivegram_values_sum

    return counter[ngram] / N

  return P

def ngram_correction(ngram, candidate_indexes = [0], n = 2):
  """
  This function corrects an n-gram by finding the most probable spelling suggestion.

  Args:
      ngram (str): The n-gram to be corrected.
      candidate_indexes (list, optional): The indexes of words in the n-gram to consider for correction (defaults to [0]).
      n (int, optional): The n-gram size (defaults to 2).

  Returns:
      str: The corrected n-gram.
  """
  global all_words
  if n == 2:
    all_words = all_bigram_words
  elif n == 5:
    all_words = all_fivegram_words

  P = get_ngram_P(n)

  best_ngram = max(ngram_candidates(ngram.lower(), candidate_indexes), key=P)
  return best_ngram

def get_new_ngram_candidates(ngram_candidates, word):
  """
  This function generates new n-gram candidates by appending a word.

  Args:
      ngram_candidates (set): The current set of n-gram candidates.
      word (str): The word to be appended.

  Returns:
      set: The updated set of n-gram candidates.
  """
  if (len(ngram_candidates) == 0):
    return {word,}

  new_ngram_candidates = {ngram_candidate + ' ' + word for ngram_candidate in ngram_candidates}
  return new_ngram_candidates

def ngram_candidates(ngram, candidate_indexes):
  """
  This function generates all possible n-gram candidates with corrections for specified words.

  Args:
      ngram (str): The n-gram to generate candidates for.
      candidate_indexes (list): The indexes of words in the n-gram to consider for correction.

  Returns:
      set: A set of all possible corrected n-gram candidates.
  """
  words = ngram.split(' ')
  ngram_candidates = set()

  for i, word in enumerate(words):
    new_ngram_candidates = set()
    if i in candidate_indexes:
      word_candidates = candidates(word)
      for word_candidate in word_candidates:
        new_ngram_candidates |= get_new_ngram_candidates(ngram_candidates, word_candidate)
    else:
      new_ngram_candidates |= get_new_ngram_candidates(ngram_candidates, word)
    ngram_candidates = new_ngram_candidates

  return ngram_candidates

def candidates(word):
  """
  This function generates possible spelling corrections for a word.

  Args:
      word (str): The word to generate corrections for.

  Returns:
      set: A set of possible corrections for the word, including the original word.
  """
  return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
  """
  This function filters a list of words and returns only those present in the all_words set.

  Args:
      words (list): A list of words to filter.

  Returns:
      set: A set containing only the words from the input list that are present in the all_words set.
  """
  return set(w for w in words if w in all_words)

def edits1(word):
    "All edits that are one edit away from `word`."
    all_characters = r'abcdefghijklmnopqrstuvwxyz1234567890!@#$%^&*()-=_+[]{}|;:,.<>?/~`"\''
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in all_characters]
    inserts = [L + c + R for L, R in splits for c in all_characters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
  """
  This function generates all edits that are one edit away from a given word.

  Args:
      word (str): The word to generate edits for.

  Returns:
      set: A set containing all possible edits with single character insertions, deletions, replacements, or transpositions.
  """
  return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def split_into_ngrams(text, n = 2):
  """
  This function splits a text into n-grams.

  Args:
      text (str): The text to split.
      n (int, optional): The n-gram size (defaults to 2).

  Returns:
      list: A list of n-grams extracted from the text.
  """
  words = text.split(' ')
  ngrams = []
  for i in range(len(words) - n + 1):
      ngram = ' '.join(words[i:i+n])
      ngrams.append(ngram)
  return ngrams

def correct_text(text, n = 2):
  """
  This function corrects spelling errors in a text using n-gram analysis.

  Args:
      text (str): The text to be corrected.
      n (int, optional): The n-gram size to use for correction (defaults to 2).

  Returns:
      str: The corrected text.
  """
  corrected_words = []
  words = text.split()

  # Calculate the total number of n-grams in the text
  ngrams_amount = len(words) - n + 1

  # Determine how many bigrams to process based on n
  bigrams_until = max(2, n-1)

  # Correct the first bigram
  bigram = ' '.join(words[0:2])
  correction = ngram_correction(bigram, [0, 1], n=2)
  correction_array = correction.split()
  corrected_words += correction_array

  # Correct bigrams until n-1 or 2 if ngram is a bigram
  for i in range(1, bigrams_until-1):
    bigram = ' '.join([corrected_words[i], words[i+1]])
    correction = ngram_correction(bigram, [1], n=2)
    correction_array = correction.split()
    corrected_words.append(correction_array[1])

  # Correct remaining words using n-grams (from bigrams_until to the end)
  for i in range(bigrams_until, n-1 + ngrams_amount):
    index = n-1

    # Create the n-gram for correction
    ngram = ' '.join(corrected_words[i-n+1:i] + [words[i]])
    correction = ngram_correction(ngram, [index], n=n)
    correction_array = correction.split()
    corrected_words.append(correction_array[index])

  return ' '.join(corrected_words)


In [18]:
correct_text('hdw c0n i0 not truot you if i sincerly lovee dyou', n=2)

'how can i not trust you if i sincerely lovie you'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

1. Ngram Dataset:

  The code uses separate counters for bigrams and fivegrams and pottentially any ngrams. This is a reasonable choice because:
    * Bigrams are more common and computationally cheaper to store and process than larger n-grams.
    * ngrams might capture more context for rarer spelling mistakes but come at a storage and processing cost
    
  Ngraws were built from a the data given. Specifically:
    * Bigrams use combination data from `bigrams.txt` and `coca_all_links.txt`
    * Fivegrams use only data from `fivegrams.txt`

2. Weights for edit distance:

  The code uses the known function to check if a word exists in the vocabulary. This implies assigning higher weight to words with no edits (edit distance 0).
  It then checks for edits1 (one edit away) and edits2 (two edits away) but doesn't explicitly assign weights. A common approach is to assign a higher weight to edits1 than edits2, reflecting the assumption that typos are more likely to be single-character errors.

3. Beam Search Parameters:

  The code doesn't implement beam search explicitly. It explores all possible candidate corrections for each n-gram and chooses the one with the highest probability based on the n-gram model.

4. Lowercase Conversion:
  The code converts the input n-gram to lowercase before looking it up in the n-gram counters (ngram.lower()) and generating correction candidates. This is a common choice because many spelling mistakes are not case-sensitive. It simplifies the process and reduces the number of n-grams to store and compare.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

## Pre-computed n-grams

The code utilizes pre-computed n-gram statistics like bigrams_counter and bigram_values_sum. This is a performance optimization. Calculating n-gram probabilities on the fly for every correction would be computationally expensive. Pre-computing these values allows for faster lookups during the correction process.

## Focus on bigrams for initial correction

The correct_text function prioritizes bigram correction for the first few words in the sentence until it will be possible to use ngram. This is because bigrams are often more reliable indicators of spelling errors compared to higher-order n-grams, which can be more sparse in a given corpus. Also it takes more computations to compute n-grams on initial stages until n-1 words are already computed

## Flexible n-gram correction:
The ngram_correction function considers corrections for all words within the n-gram specified by the candidate_indexes argument. This allows for targeted correction based on potential errors in specific words. This also allows different combinations of words to be corected simultaneously that could provide better accuracy and more cohesive outputs

test set taken from here
https://www.kaggle.com/datasets/samarthagarwal23/spelling-mistake-data-1mn

In [19]:
TEST_PATH = os.path.join(ROOT_DIR, 'test.csv')

In [20]:
test_df = pd.read_csv(TEST_PATH)

In [21]:
test_df.head()

,text,augmented_text
0,project looks to mulesing genetic alternative,project looks to muelsnig ngeetic alternative
1,chemical agents used during protest at port au...,chemical agents used during LrotWst at port ah...
2,business chamber seeks budget infrastructure b...,business hcmaber seeks budget infrastrcutuer b...
3,3600 trips made to darwin tip after cyclone,3600 trips made to adrwni tip after cyconle
4,go between bridge to open july 5,go net3een brisye to lprn july 5


In [32]:
from tqdm import tqdm

def evaluate_correction(corrector, test_data, n=2, sample_size=100):
  """
  Evaluates the performance of a spell correction function.

  Args:
      corrector: The function to evaluate.
      test_data: A pandas dataframe containing columns 'text' (original text)
                 and 'augmented_text' (text with errors).
      n: The n-gram size to use for correction (default 2).
      sample_size: Number of samples to use from the test data (default 100).

  Returns:
      A tuple containing word accuracy and sentence accuracy.
  """

  total_words, correct_words = 0, 0
  total_sentences, correct_sentences = 0, 0

  # Limit sample size to dataframe length
  limited_data = test_data.iloc[:min(sample_size, len(test_data))]

  # Iterate through limited data using tqdm progress bar
  for _, row in tqdm(limited_data.iterrows(), total=sample_size):
    original_text, augmented_text = row['text'], row['augmented_text']

    corrected_text = corrector(augmented_text, n) if n > 1 else corrector(augmented_text)

    original_words, augmented_words, corrected_words = map(str.split, (original_text, augmented_text, corrected_text))

    total_sentences += 1
    if original_text == corrected_text:
      correct_sentences += 1

    for orig, aug, corr in zip(original_words, augmented_words, corrected_words):
      if orig != aug:
        total_words += 1
        if orig == corr:
          correct_words += 1


  # Calculate accuracy metrics
  word_accuracy = correct_words / total_words if total_words else 0
  sentence_accuracy = correct_sentences / total_sentences if total_sentences else 0

  return word_accuracy, sentence_accuracy

In [33]:
word_accuracy, sentence_accuracy = evaluate_correction(norvig_correct_text, test_df, n=1, sample_size=500)
print()
print(f"Metrics for norvig correction")
print("\tWord Accuracy:", word_accuracy)
print("\tSentence Accuracy:", sentence_accuracy)

100%|██████████| 500/500 [02:06<00:00,  3.97it/s]


Metrics for norvig correction
	Word Accuracy: 0.43536875495638383
	Sentence Accuracy: 0.094


In [34]:
word_accuracy, sentence_accuracy = evaluate_correction(correct_text, test_df, n=2, sample_size=500)
print()
print(f"Metrics for bigram correction")
print("\tWord Accuracy:", word_accuracy)
print("\tSentence Accuracy:", sentence_accuracy)

100%|██████████| 500/500 [10:19<00:00,  1.24s/it]


Metrics for bigram correction
	Word Accuracy: 0.4528152260111023
	Sentence Accuracy: 0.138


In [35]:
word_accuracy, sentence_accuracy = evaluate_correction(correct_text, test_df, n=5, sample_size=500)
print()
print(f"Metrics for fivegram correction")
print("\tWord Accuracy:", word_accuracy)
print("\tSentence Accuracy:", sentence_accuracy)

100%|██████████| 500/500 [10:42<00:00,  1.28s/it]


Metrics for fivegram correction
	Word Accuracy: 0.43061062648691517
	Sentence Accuracy: 0.122


Bigram-based solution has the greatest scores for the chosen dataset